<a href="https://colab.research.google.com/github/ColeJMitchell/machine-learning-stock-prediction/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages

In [2]:
import pandas as pd

Displaying The Data

In [5]:
data = pd.read_parquet("wallstreetbets-collection.parquet")
print(data.head)

# Remove extraneous fields
fields = ('title',
          'created_utc',
          'id',
          'is_original_content',
          'link_flair_text',
          'locked',
          'name',
          'num_comments',
          'over_18',
          'permalink',
          'selftext',
          'spoiler',
          'upvote_ratio')
list_of_submissions = []

# Parse each submission into a dictionary of the lised fields.
for submission in all_posts:
    full = vars(submission)
    sub_dict = {field:full[field] for field in fields}
    list_of_submissions.append(sub_dict)

# Create a python dataframe of these submissions.
collected_data = pd.DataFrame.from_records(list_of_submissions)

# Display the dataframe.
display(collected_data)



<bound method NDFrame.head of                                                   title   created_utc  \
0                                       Tariffs bonanza  1.743624e+09   
1                                                     🤔  1.743624e+09   
2     Is It Just Me, or Is Robinhood’s Security Stil...  1.743623e+09   
3     200 puts (~10k): If NVDA goes to $90 this week...  1.743393e+09   
4                                        All in on Hood  1.743015e+09   
...                                                 ...           ...   
1338  EU may delay first counter-tariffs against U.S...  1.742469e+09   
1339  US companies race to secure import tariff exem...  1.742469e+09   
1340  Tesla to recall more than 46,000 Cybertrucks d...  1.742466e+09   
1341         Daily Discussion Thread for March 20, 2025  1.742465e+09   
1342                 Stop or GO for GO?(grocery outlet)  1.742464e+09   

           id  is_original_content   link_flair_text  locked        name  \
0     1jpyan4    